# General Call Price Evaluation 

## Get Prices

In [1]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for which you want the options chain
ticker_symbol = "CIFR"

# Create a Ticker object using yfinance
ticker = yf.Ticker(ticker_symbol)

# Get options expiration dates
options_expirations = ticker.options

# Initialize an empty DataFrame for the options data
df = pd.DataFrame()

# Loop through each expiration date and collect options data
for expiration in options_expirations:
    # Get the option chain for the given expiration
    opt_chain = ticker.option_chain(expiration)

    # Extract puts 
    puts = opt_chain.puts

    # Add an 'Expiration' column to puts 
    puts['Expiration'] = expiration

    # Append the data to the options_data DataFrame
    df = pd.concat([df, puts], ignore_index=True)

# Convert 'Expiration' to datetime format and optionally format it
df['Date'] = pd.to_datetime(df['Expiration']).dt.strftime('%Y-%m-%d')

# Display the first few rows of the DataFrame
df


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,Expiration,Date
0,CIFR240315P00001000,2024-03-04 16:41:19+00:00,1.0,0.04,0.00,0.05,0.00,0.000000,1.0,58.0,7.750000,0.0,REGULAR,USD,2024-03-15,2024-03-15
1,CIFR240315P00001500,2024-02-13 16:40:33+00:00,1.5,0.01,0.00,0.30,0.00,0.000000,2.0,35.0,8.625005,0.0,REGULAR,USD,2024-03-15,2024-03-15
2,CIFR240315P00002000,2024-03-11 13:30:01+00:00,2.0,0.03,0.00,0.10,0.00,0.000000,2.0,6446.0,4.062505,0.0,REGULAR,USD,2024-03-15,2024-03-15
3,CIFR240315P00002500,2024-03-11 15:57:37+00:00,2.5,0.05,0.00,0.05,0.03,150.000020,1.0,2030.0,1.968750,0.0,REGULAR,USD,2024-03-15,2024-03-15
4,CIFR240315P00003000,2024-03-12 19:28:51+00:00,3.0,0.05,0.05,0.10,-0.05,-50.000000,151.0,1940.0,1.187504,0.0,REGULAR,USD,2024-03-15,2024-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,CIFR260116P00004500,2024-02-26 18:53:42+00:00,4.5,3.16,2.75,3.60,0.00,0.000000,6.0,57.0,1.357425,True,REGULAR,USD,2026-01-16,2026-01-16
87,CIFR260116P00005000,2024-02-27 16:12:40+00:00,5.0,3.50,3.20,3.90,0.00,0.000000,40.0,82.0,1.312503,True,REGULAR,USD,2026-01-16,2026-01-16
88,CIFR260116P00005500,2024-03-05 14:53:47+00:00,5.5,4.00,3.60,4.40,0.00,0.000000,3.0,193.0,1.318363,True,REGULAR,USD,2026-01-16,2026-01-16
89,CIFR260116P00007000,2024-01-22 15:46:23+00:00,7.0,5.05,3.80,6.60,0.00,0.000000,2.0,15.0,1.224613,True,REGULAR,USD,2026-01-16,2026-01-16


## Date of Interest

In [2]:
target_date = '2026-01-16'

filtered_df = df[df['Date'] == target_date]

# Sort by 'Strike' column from lowest to highest
filtered_df = filtered_df.sort_values(by='strike')

# Convert filtered DataFrame to list of tuples (Strike, Ask Price)
options = list(filtered_df[['strike', 'ask']].dropna().apply(tuple, axis=1))

print("Options for", target_date, ":", options)

Options for 2026-01-16 : [(1.0, 1.55), (1.5, 1.2), (2.0, 3.7), (2.5, 1.9), (3.0, 2.3), (3.5, 2.7), (4.0, 3.0), (4.5, 3.6), (5.0, 3.9), (5.5, 4.4), (7.0, 6.6), (10.0, 8.4)]


## Data & Params

In [3]:
# All options expire on January 16th 2026

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

# Retrieve historical data
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')

data = yf.download(ticker_symbol, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()

daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

[*********************100%%**********************]  1 of 1 completed
C:\Users\falty\AppData\Local\Temp\ipykernel_17216\4211646061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


## Profit Based on Final Stock Price

In [4]:
# BTCC-B.TO Call Option Price Evaluation

risk_free_rate = 0.045
initial_capital = 5_000
current_stock_price = data['Close'].iloc[-1]  
final_stock_price = current_stock_price * 10
num_shares_per_option = 100

# Function to calculate profit for buying and holding the stock
def calculate_stock_profit(initial_capital, current_price, final_price):
    num_shares = initial_capital // current_price
    return num_shares * (final_price - current_price)

# Function to calculate profit for each option scenario
def calculate_option_profit(initial_capital, strike, ask_price, final_price):
    num_options = initial_capital // (ask_price * num_shares_per_option)
    profit_per_option = max(0, final_price - strike) - ask_price
    return num_options * profit_per_option * num_shares_per_option

# Print final stock price
print(f"Final stock price: ${final_stock_price:.2f}")

# Calculate profit for buying and holding
stock_profit = calculate_stock_profit(initial_capital, current_stock_price, final_stock_price)
print(f"Profit from buying and holding the stock: ${stock_profit:.2f}")

# Calculate and display profit for each option scenario
for strike, ask_price in options:
    option_profit = calculate_option_profit(initial_capital, strike, ask_price, final_stock_price)
    print(f"Profit from option with strike ${strike} and ask price ${ask_price}: ${option_profit:.2f}")


Final stock price: $31.60
Profit from buying and holding the stock: $44992.08
Profit from option with strike $1.0 and ask price $1.55: $92960.00
Profit from option with strike $1.5 and ask price $1.2: $118490.00
Profit from option with strike $2.0 and ask price $3.7: $33670.00
Profit from option with strike $2.5 and ask price $1.9: $70720.00
Profit from option with strike $3.0 and ask price $2.3: $55230.00
Profit from option with strike $3.5 and ask price $2.7: $45720.00
Profit from option with strike $4.0 and ask price $3.0: $39360.00
Profit from option with strike $4.5 and ask price $3.6: $30550.00
Profit from option with strike $5.0 and ask price $3.9: $27240.00
Profit from option with strike $5.5 and ask price $4.4: $23870.00
Profit from option with strike $7.0 and ask price $6.6: $12600.00
Profit from option with strike $10.0 and ask price $8.4: $6600.00


## Binomial Tree

In [5]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.stats import norm

# Assuming data['Daily_Return'] contains the daily returns
historical_volatility = np.std(np.log(1 + data['Daily_Return'].dropna())) * np.sqrt(252)  # Annualized using trading days in a year
mean_return = np.mean(data['Daily_Return'].dropna()) * 252  # Annualized

historical_skewness = skew(data['Daily_Return'].dropna())
historical_kurtosis = kurtosis(data['Daily_Return'].dropna(), fisher=False)

def binomial_tree_put_option_price(S, K, T_days, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0, drift=0):
    T = T_days / 252  # Convert days to years
    dt = T / N
    u = np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt))  # Incorporating drift
    d = 1 / u
    p = (np.exp((r - dividends) * dt) - d) / (u - d)

    p += skew * 0.001 * (1 - 2 * p) + kurtosis * 0.0005 * (1 - 2 * p)
    p = min(max(p, 0), 1)

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(K - prices, 0)  # Changed for put option

    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        # For American options, compare with early exercise
        option_values[:i+1] = np.maximum(option_values[:i+1], K - prices[:i+1])  # Changed for put option

    return option_values[0]

### Price Specific Date

In [7]:
# Example usage:
risk_free_rate = 0.05  # Example rate
T_days = 6  # days until expiration
N = 100  # number of steps

# Example usage
for strike, ask_price in options:
    option_price = binomial_tree_put_option_price(
        current_stock_price, strike, T_days, risk_free_rate, historical_volatility, N,
        dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
    )
    print(f"Call Option Price for strike {strike}: ${option_price:.2f}")

Call Option Price for strike 1.0: $0.44
Call Option Price for strike 1.5: $0.94
Call Option Price for strike 2.0: $1.44
Call Option Price for strike 2.5: $1.94
Call Option Price for strike 3.0: $2.44
Call Option Price for strike 3.5: $2.94
Call Option Price for strike 4.0: $3.44
Call Option Price for strike 4.5: $3.94
Call Option Price for strike 5.0: $4.44
Call Option Price for strike 5.5: $4.94
Call Option Price for strike 7.0: $6.44
Call Option Price for strike 10.0: $9.44


### Look for Undervalued puts

In [8]:
N = 5_000

for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = binomial_tree_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility, N,
            dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
        )

        if ask_price < option_price:
            print(f"Underpriced Call Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")

TypeError: unsupported operand type(s) for -: 'str' and 'Timestamp'

## Monte-Carlo Simulation 

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from numba import jit, prange

# Calculate log returns
log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

# Identify jumps
jump_threshold = 3 * np.std(log_returns)  # Example threshold
jumps = log_returns[abs(log_returns) > jump_threshold]

# Estimate lambda (jump intensity)
lambda_ = len(jumps) / (len(log_returns) / 365)  # Number of jumps per year

# Estimate mu_J and sigma_J (jump size parameters)
# We use the jumps themselves, not the ratio to their lagged values
mu_J = np.mean(jumps)
sigma_J = np.std(jumps)

@jit(nopython=True, parallel=True)  # Enable JIT compilation with parallel execution
def monte_carlo_put_option_price(S, K, T_months, r, sigma, lambda_, mu_J, sigma_J, num_paths=10000, num_steps=100, skew=0, kurtosis=0):
    """
    Monte Carlo simulation for European put option pricing with jump diffusion.

    Parameters:
    S (float): Current stock price.
    K (float): Strike price of the option.
    T_months (int): Time to maturity in months.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the underlying asset.
    lambda_ (float): Jump intensity, representing the frequency of jumps.
    mu_J (float): Mean of the logarithm of the jump size.
    sigma_J (float): Standard deviation of the logarithm of the jump size.
    num_paths (int): Number of simulated paths in the Monte Carlo simulation.
    num_steps (int): Number of time steps in each simulated path.
    skew (float): Skewness of the asset returns. Defaults to 0 for no skew.
    kurtosis (float): Kurtosis of the asset returns. Defaults to 0 for normal kurtosis.

    Returns:
    float: Estimated price of the European put option.
    """
    T = T_months / 365  # Convert maturity from months to years
    dt = T / num_steps  # Time step for simulation
    discount_factor = np.exp(-r * T)  # Discount factor for present value

    payoffs = np.zeros(num_paths)  # Initialize array for option payoffs
    for i in prange(num_paths):  # Parallel loop for each path
        S_t = S  # Initial stock price for this path
        for j in range(num_steps):  # Time step loop
            z = np.random.normal()  # Standard normal random variable
            shock = z + skew * (z ** 2 - 1)  # Adjust random variable for skew and kurtosis
            num_jumps = np.random.poisson(lambda_ * dt)  # Poisson process for number of jumps
            jump_sum = np.sum(np.random.lognormal(mu_J, sigma_J, num_jumps) - 1)  # Sum of jump sizes
            S_t *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * shock + jump_sum)  # Stock price update
        payoffs[i] = max(K - S_t, 0)  # Put option payoff for this path

    return discount_factor * np.mean(payoffs)  # Average discounted payoff

### Price Specific Date

In [ ]:
# Example usage
for strike, ask_price in options:
    T_days = 6 
    option_price = monte_carlo_option_price(
        current_stock_price, strike, T_months, risk_free_rate, historical_volatility,
        lambda_, mu_J, sigma_J,
        num_paths=1_000, num_steps=T_days*24,
        skew=historical_skewness, kurtosis=historical_kurtosis
    )
    print(f"Monte Carlo Call Option Price for strike {strike}: ${option_price:.2f}")

### Look for Undervalued puts

In [ ]:
for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = monte_carlo_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility,
            lambda_, mu_J, sigma_J,
            num_paths=100_000, num_steps=T_days * 24,  # Assuming 24 hours in a day for finer granularity
            skew=historical_skewness, kurtosis=historical_kurtosis
        )

        if ask_price < option_price:
            print(f"Underpriced Call Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")